<img src="https://www.upmetropolitana.edu.mx/img/logos/logo-2.png" alt="Descripción" width="300" />

# Pipeline Robusto de Machine Learning y Validación Cruzada
## Universidad Politécnica Metropolitana de Hidalgo
### Maestría en Inteligencia Artificial
### Cuatrimestre 2 - Big Data
Profesor: Dr. Jaime Aguilar Ortiz<br>
Fecha: 17 de enero de 2026

| Matrícula | Alumno|
|------------|------------|
| 253220094     | CORTÉS SPROSS J. GERHARD     |
| 253220116     | ROMERO LORA JÉSSICA MELANI     |
| 253220020     | SANTOS MARTÍNEZ VÍCTOR MANUEL     |

## Preparación Conceptual
1. Temas que se abordarán: ingesta, normalización e integración de fuentes heterogeneas  de datos (de 3 fuentes distintas) en un flujo reproducible.
2. En qué consisten: El pipeline en este caso, consiste en aplicar conceptos básicos de Big Data como variedad, preparción y trazabilidad directa.
3. Los resultados principales podrían ser las métricas comparativas, desarrollo de un glosario de términos y la visualización que evidencie los patrones para su reproducibilidad y repetitibilidad.
4. Aplicaciones prácticas: auditoría de información, análisis de calidad de datos y generación de reportes para la toma de decisiones profesionales.

## Ingesta y registro de fuentes
El código recibe rutas de archivos (o una carpeta) y registra, para cada fuente: nombre, formato, tamaño en bytes, fecha de procesamiento y conteo básico (líneas o caracteres si aplica).

Para facilitar la tarea, las rutas de los archivos son siempre consultables desde un repositorio público de Github, en caso de la necesidad de implementar un repositorio privado en el futuro, se pueden incorporar claves de acceso.

---

### Importación de librerías y datos
Se realiza una instalación **exclusiva de python (!pip)** para asegurar que se contarán los las mínimas librerías necesarias.

In [ ]:
!pip -q install pandas matplotlib beautifulsoup4 lxml python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os, re, json, csv, time, hashlib
from datetime import datetime
from dataclasses import dataclass, asdict
from IPython.display import display, Markdown

import pandas as pd
import matplotlib.pyplot as plt

from io import StringIO

import requests
from bs4 import BeautifulSoup

from docx import Document
from docx.shared import Inches

### Definición de las rutas de acceso

In [ ]:
TXT_URL  = "https://raw.githubusercontent.com/VManuelSM/Actividad_1_Big_Data/refs/heads/main/logs_sistema.txt"
CSV_URL  = "https://github.com/VManuelSM/Actividad_1_Big_Data/raw/refs/heads/main/registros_eventos.csv"
HTML_URL = "https://github.com/VManuelSM/Actividad_1_Big_Data/raw/refs/heads/main/reporte_incidencias.html"

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("OK: variables configuradas")

OK: variables configuradas


### Funciones de utilidad

In [ ]:
def download_text(url: str) -> str:
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.text

def sha256_text(text: str) -> str:
    return hashlib.sha256(text.encode("utf-8")).hexdigest()

def normalize_text(text: str) -> str:
    text = text.lower().strip()
    # deja letras, números, espacios y acentos básicos; cambia lo demás a espacio
    text = re.sub(r"[^a-z0-9áéíóúñü\s]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

### Ingesta de TXT

In [ ]:
txt_raw = download_text(TXT_URL)
txt_hash = sha256_text(txt_raw)

txt_norm = normalize_text(txt_raw)

# Prepare data for DataFrame
txt_info_data = [
    {"Descripción": "TXT name", "Valor": TXT_URL.split("/")[-1]},
    {"Descripción": "TXT type file", "Valor": TXT_URL.split(".")[-1]},
    {"Descripción": "TXT shape", "Valor": f"{txt_raw.count('\n')} lines"},
    {"Descripción": "TXT bytes", "Valor": len(txt_raw.encode("utf-8"))},
    {"Descripción": "TXT hash", "Valor": f"{txt_hash[:16]} ..."},
    {"Descripción": "Processing date", "Valor": datetime.now().strftime("%Y-%m-%d %H:%M:%S")},
    {"Descripción": "TXT preview", "Valor": f"{txt_raw[:120]} ..."}
]

df_txt_info = pd.DataFrame(txt_info_data)
display(df_txt_info)

,Descripción,Valor
0,TXT name,logs_sistema.txt
1,TXT type file,txt
2,TXT shape,5 lines
3,TXT bytes,355
4,TXT hash,e04e6b7b2c31aee9 ...
5,Processing date,2026-01-17 21:40:45
6,TXT preview,El sistema de atención ciudadana presentó inte...


### Ingesta de csv

In [ ]:
from IPython.core.display import display_markdown
csv_raw = download_text(CSV_URL)
csv_hash = sha256_text(csv_raw)

df_csv = pd.read_csv(StringIO(csv_raw))

# Prepare data for DataFrame in a two-column format
csv_info_data = [
    {"Descripción": "CSV name", "Valor": CSV_URL.split("/")[-1]},
    {"Descripción": "CSV type file", "Valor": CSV_URL.split(".")[-1]},
    {"Descripción": "CSV shape", "Valor": f"{df_csv.shape[0]} rows, {df_csv.shape[1]} columns"},
    {"Descripción": "CSV bytes", "Valor": len(csv_raw.encode("utf-8"))},
    {"Descripción": "CSV hash", "Valor": f"{csv_hash[:16]} ..."},
    {"Descripción": "Processing date", "Valor": datetime.now().strftime("%Y-%m-%d %H:%M:%S")},
    {"Descripción": "CSV preview", "Valor": f"{df_csv.to_string(index=False)[:120]} ..."} # Previews first 120 chars of string representation
]

display(Markdown('### Descripción del CSV'))
df_csv_info = pd.DataFrame(csv_info_data)
display(df_csv_info)
display(Markdown('---'))
df_csv


### Descripción del CSV

,Descripción,Valor
0,CSV name,registros_eventos.csv
1,CSV type file,csv
2,CSV shape,"5 rows, 4 columns"
3,CSV bytes,278
4,CSV hash,6e1f53eac20e24d6 ...
5,Processing date,2026-01-17 21:47:39
6,CSV preview,id_evento fecha categoria ...


---

,id_evento,fecha,categoria,descripcion
0,1,2026-01-10,ERROR,Falla en autenticación de usuario
1,2,2026-01-10,WARNING,Respuesta lenta del servidor
2,3,2026-01-11,INFO,Sistema iniciado correctamente
3,4,2026-01-11,ERROR,Formulario no enviado
4,5,2026-01-12,INFO,Ajustes aplicados al sistema


### Ingesta de HTML

In [ ]:
html_raw = download_text(HTML_URL)
html_hash = sha256_text(html_raw)

soup = BeautifulSoup(html_raw, "lxml")
html_text = soup.get_text(separator=" ", strip=True)
html_norm = normalize_text(html_text)

print("HTML bytes:", len(html_raw.encode("utf-8")))
print("HTML hash :", html_hash[:16], "...")
print("HTML text preview:", html_text[:140], "...")

HTML bytes: 705
HTML hash : 7afc4a6ba7bcb1d3 ...
HTML text preview: Reporte de Incidencias Reporte de Incidencias del Sistema Durante el mes de enero se presentaron diversas incidencias relacionadas
        c ...


###Sistema de auditoría y trazabilidad (SHA-256)
---

In [3]:
import os
import re
import json
import hashlib
import logging
from datetime import datetime
from dataclasses import dataclass, asdict, field
from typing import Dict, List, Optional, Any
from enum import Enum
from io import StringIO

import pandas as pd
import requests
from bs4 import BeautifulSoup

Configuración del sistema de entrada: logging

In [5]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('audit_pipeline.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

Enumeraciones para tipos y constantes comunes:

In [6]:
class SourceType(Enum):
  """Tipos de fuentes de datos soportadas"""
  TXT = "txt"
  CSV = "csv"
  HTML = "html"
  JSON = "json"
  XML = "xml"
class AuditStatus(Enum):
  """Estados de auditoría"""
  SUCCESS = "success"
  FAILURE = "failure"
  WARNING = "warning"
  PENDING = "pending"
  SKIPPED = "skipped"

###Configuración de URLS:

In [8]:
data_sources = {
    "TXT":"https://raw.githubusercontent.com/VManuelSM/Actividad_1_Big_Data/refs/heads/main/logs_sistema.txt",
    "CSV":"https://github.com/VManuelSM/Actividad_1_Big_Data/raw/refs/heads/main/registros_eventos.csv",
    "HTML":"https://github.com/VManuelSM/Actividad_1_Big_Data/raw/refs/heads/main/reporte_incidencias.html"
}

Output_dir = "outputs"
audit_dir = "audit_logs"

### Creación de directorios:

In [ ]:
os.makedirs(Output_dir, exist_ok=True)
os.makedirs(audit_dir, exist_ok=True)

### Clases de datos para metadatos y registros de auditoría

In [9]:
@dataclass
class HashMetadata:
  """Metadatos de integridad criptográfica"""
  sha256: str
  md5: str
  timestamp: str
  algorithm_version: str = "SHA- 256, MD5"

  def to_dict(self) -> Dict[str, str]:
    return asdict(self)

In [10]:
@dataclass
class DataQualityMetrics:
    """Métricas de calidad de datos"""
    total_records: int
    null_count: int
    duplicate_count: int
    data_types: Dict[str, str]
    completeness_ratio: float

    def to_dict(self) -> Dict[str, Any]:
        return asdict(self)

In [11]:
@dataclass
class SourceMetadata:
    """Metadatos completos de la fuente de datos"""
    source_id: str
    source_name: str
    source_type: str
    source_url: str
    size_bytes: int
    encoding: str
    processing_timestamp: str
    hash_metadata: HashMetadata
    quality_metrics: Optional[DataQualityMetrics] = None
    additional_info: Dict[str, Any] = field(default_factory=dict)

    def to_dict(self) -> Dict[str, Any]:
        data = asdict(self)
        data['hash_metadata'] = self.hash_metadata.to_dict()
        if self.quality_metrics:
            data['quality_metrics'] = self.quality_metrics.to_dict()
        return data

In [22]:
@dataclass
class AuditRecord:
    """Registro completo de auditoría"""
    audit_id: str
    pipeline_version: str
    execution_timestamp: str
    sources_processed: List[SourceMetadata]
    audit_status: str
    total_sources: int
    successful_sources: int
    failed_sources: int
    warnings: List[str] = field(default_factory=list)
    errors: List[str] = field(default_factory=list)
    environment_info: Dict[str, str] = field(default_factory=dict)

    def to_dict(self) -> Dict[str, Any]:
        data = asdict(self)
        data['sources_processed'] = [s.to_dict() for s in self.sources_processed]
        return data

    def save_to_json(self, filepath: str) -> None:
        """Guarda el registro de auditoría en formato JSON"""
        os.makedirs(os.path.dirname(filepath), exist_ok=True) # Ensure directory exists
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(self.to_dict(), f, indent=2, ensure_ascii=False)
        logger.info(f"Auditoría guardada en: {filepath}")

### Utilidades criptográficas:

In [13]:
class IntegrityValidator:
    """Validador de integridad de datos"""

    @staticmethod
    def compute_hashes(data: str) -> HashMetadata:
        """
        Calcula múltiples hashes para redundancia

        Args:
            data: Contenido a hashear

        Returns:
            HashMetadata con SHA-256 y MD5
        """
        data_bytes = data.encode('utf-8')

        sha256_hash = hashlib.sha256(data_bytes).hexdigest()
        md5_hash = hashlib.md5(data_bytes).hexdigest()

        return HashMetadata(
            sha256=sha256_hash,
            md5=md5_hash,
            timestamp=datetime.now().isoformat()
        )

    @staticmethod
    def verify_integrity(data: str, expected_sha256: str) -> bool:
        """
        Verifica la integridad de los datos

        Args:
            data: Contenido a verificar
            expected_sha256: Hash SHA-256 esperado

        Returns:
            True si la integridad es válida
        """
        current_hash = hashlib.sha256(data.encode('utf-8')).hexdigest()
        return current_hash == expected_sha256

### Procesadores de fuentes:

In [15]:
class DataSourceProcessor:
    """Procesador genérico de fuentes de datos"""

    def __init__(self, timeout: int = 30):
        self.timeout = timeout
        self.session = requests.Session()

    def download_source(self, url: str) -> str:
        """
        Descarga contenido desde una URL

        Args:
            url: URL de la fuente

        Returns:
            Contenido como texto
        """
        try:
            response = self.session.get(url, timeout=self.timeout)
            response.raise_for_status()
            logger.info(f"Descarga exitosa: {url}")
            return response.text
        except requests.exceptions.RequestException as e:
            logger.error(f"Error descargando {url}: {e}")
            raise

    def process_txt(self, url: str, source_id: str) -> SourceMetadata:
        """Procesa archivo TXT"""
        logger.info(f"Procesando TXT: {source_id}")

        raw_content = self.download_source(url)
        hash_meta = IntegrityValidator.compute_hashes(raw_content)

        lines = raw_content.split('\n')

        metadata = SourceMetadata(
            source_id=source_id,
            source_name=url.split('/')[-1],
            source_type=SourceType.TXT.value,
            source_url=url,
            size_bytes=len(raw_content.encode('utf-8')),
            encoding='utf-8',
            processing_timestamp=datetime.now().isoformat(),
            hash_metadata=hash_meta,
            additional_info={
                'total_lines': len(lines),
                'non_empty_lines': len([l for l in lines if l.strip()]),
                'total_characters': len(raw_content)
            }
        )

        return metadata

    def process_csv(self, url: str, source_id: str) -> SourceMetadata:
        """Procesa archivo CSV con métricas de calidad"""
        logger.info(f"Procesando CSV: {source_id}")

        raw_content = self.download_source(url)
        hash_meta = IntegrityValidator.compute_hashes(raw_content)

        # Cargar en DataFrame para análisis
        df = pd.read_csv(StringIO(raw_content))

        # Métricas de calidad
        quality_metrics = DataQualityMetrics(
            total_records=len(df),
            null_count=int(df.isnull().sum().sum()),
            duplicate_count=int(df.duplicated().sum()),
            data_types={col: str(dtype) for col, dtype in df.dtypes.items()},
            completeness_ratio=float(1 - (df.isnull().sum().sum() / df.size))
        )

        metadata = SourceMetadata(
            source_id=source_id,
            source_name=url.split('/')[-1],
            source_type=SourceType.CSV.value,
            source_url=url,
            size_bytes=len(raw_content.encode('utf-8')),
            encoding='utf-8',
            processing_timestamp=datetime.now().isoformat(),
            hash_metadata=hash_meta,
            quality_metrics=quality_metrics,
            additional_info={
                'rows': len(df),
                'columns': len(df.columns),
                'column_names': list(df.columns)
            }
        )

        return metadata

    def process_html(self, url: str, source_id: str) -> SourceMetadata:
        """Procesa archivo HTML"""
        logger.info(f"Procesando HTML: {source_id}")

        raw_content = self.download_source(url)
        hash_meta = IntegrityValidator.compute_hashes(raw_content)

        # Parsear HTML
        soup = BeautifulSoup(raw_content, 'lxml')
        text_content = soup.get_text(separator=' ', strip=True)

        metadata = SourceMetadata(
            source_id=source_id,
            source_name=url.split('/')[-1],
            source_type=SourceType.HTML.value,
            source_url=url,
            size_bytes=len(raw_content.encode('utf-8')),
            encoding='utf-8',
            processing_timestamp=datetime.now().isoformat(),
            hash_metadata=hash_meta,
            additional_info={
                'html_tags_count': len(soup.find_all()),
                'text_length': len(text_content),
                'has_title': bool(soup.title),
                'title': soup.title.string if soup.title else None
            }
        )

        return metadata

### Motor de auditoría

In [19]:
class AuditEngine:
    """Motor principal de auditoría y trazabilidad"""

    def __init__(self, pipeline_version: str = "1.0.0"):
        self.pipeline_version = pipeline_version
        self.processor = DataSourceProcessor()
        self.audit_id = self._generate_audit_id()

    def _generate_audit_id(self) -> str:
        """Genera un ID único para la auditoría"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        random_suffix = hashlib.sha256(str(datetime.now()).encode()).hexdigest()[:8]
        return f"AUDIT_{timestamp}_{random_suffix}"

    def _get_environment_info(self) -> Dict[str, str]:
        """Captura información del entorno de ejecución"""
        import sys
        import platform

        return {
            'python_version': sys.version,
            'platform': platform.platform(),
            'machine': platform.machine(),
            'processor': platform.processor(),
            'execution_user': os.getenv('USER', 'unknown')
        }

    def run_audit(self, sources: Dict[str, str]) -> AuditRecord:
        """
        Ejecuta auditoría completa de todas las fuentes

        Args:
            sources: Diccionario {source_id: url}

        Returns:
            AuditRecord con resultados completos
        """
        logger.info(f"Iniciando auditoría: {self.audit_id}")

        processed_sources = []
        successful = 0
        failed = 0
        warnings = []
        errors = []

        for source_id, url in sources.items():
            try:
                # Determinar tipo de procesador
                extension = url.split('.')[-1].lower()

                if extension == 'txt':
                    metadata = self.processor.process_txt(url, source_id)
                elif extension == 'csv':
                    metadata = self.processor.process_csv(url, source_id)
                elif extension in ['html', 'htm']:
                    metadata = self.processor.process_html(url, source_id)
                else:
                    warnings.append(f"Tipo no soportado para {source_id}: {extension}")
                    continue

                processed_sources.append(metadata)
                successful += 1

                # Verificar calidad de datos si aplica
                if hasattr(metadata, 'quality_metrics') and metadata.quality_metrics:
                    if metadata.quality_metrics.completeness_ratio < 0.95:
                        warnings.append(
                            f"{source_id}: Completitud baja ({metadata.quality_metrics.completeness_ratio:.2%})"
                        )

            except Exception as e:
                failed += 1
                error_msg = f"Error procesando {source_id}: {str(e)}"
                errors.append(error_msg)
                logger.error(error_msg)

        # Determinar estado general
        if failed == 0:
            status = AuditStatus.SUCCESS.value
        elif successful > 0:
            status = AuditStatus.WARNING.value
        else:
            status = AuditStatus.FAILURE.value

        # Crear registro de auditoría
        audit_record = AuditRecord(
            audit_id=self.audit_id,
            pipeline_version=self.pipeline_version,
            execution_timestamp=datetime.now().isoformat(),
            sources_processed=processed_sources,
            audit_status=status,
            total_sources=len(sources),
            successful_sources=successful,
            failed_sources=failed,
            warnings=warnings,
            errors=errors,
            environment_info=self._get_environment_info()
        )

        # Guardar auditoría
        audit_filepath = os.path.join(
            audit_dir,
            f"{self.audit_id}.json"
        )
        audit_record.save_to_json(audit_filepath)

        # Generar reporte en consola
        self._print_audit_summary(audit_record)

        return audit_record

    def _print_audit_summary(self, audit: AuditRecord) -> None:
        """Imprime resumen de auditoría en consola"""
        print("\n" + "="*80)
        print(f"RESUMEN DE AUDITORÍA: {audit.audit_id}")
        print("="*80)
        print(f"Estado: {audit.audit_status.upper()}")
        print(f"Fuentes totales: {audit.total_sources}")
        print(f"Exitosas: {audit.successful_sources}")
        print(f"Fallidas: {audit.failed_sources}")
        print(f"Advertencias: {len(audit.warnings)}")
        print(f"Errores: {len(audit.errors)}")
        print("\nFuentes procesadas:")
        for source in audit.sources_processed:
            print(f"  ✓ {source.source_id} ({source.source_type})")
            print(f"    SHA-256: {source.hash_metadata.sha256}")
            print(f"    Tamaño: {source.size_bytes:,} bytes")
        print("="*80 + "\n")

### Ejecución principal

In [23]:
def main():
    """Función principal de ejecución"""
    print("Iniciando Sistema de Auditoría y Trazabilidad")
    print("Universidad Politécnica Metropolitana de Hidalgo\n")

    # Crear motor de auditoría:
    engine = AuditEngine(pipeline_version="1.0.0")

    # Ejecutar auditoría completa
    audit_result = engine.run_audit(data_sources)

    # Generar DataFrame con resumen
    summary_data = []
    for source in audit_result.sources_processed:
        summary_data.append({
            'Source ID': source.source_id,
            'Type': source.source_type,
            'Size (bytes)': source.size_bytes,
            'SHA-256': source.hash_metadata.sha256[:16] + '...',
            'Timestamp': source.processing_timestamp
        })

    df_summary = pd.DataFrame(summary_data)
    print("\nResumen de Fuentes Procesadas:")
    print(df_summary.to_string(index=False))

    return audit_result


if __name__ == "__main__":
    audit_result = main()

Iniciando Sistema de Auditoría y Trazabilidad
Universidad Politécnica Metropolitana de Hidalgo


RESUMEN DE AUDITORÍA: AUDIT_20260119_004313_e623fa28
Estado: SUCCESS
Fuentes totales: 3
Exitosas: 3
Fallidas: 0
Advertencias: 0
Errores: 0

Fuentes procesadas:
  ✓ TXT (txt)
    SHA-256: e04e6b7b2c31aee9b69333443273da773ff4c8490757501619f396499d356267
    Tamaño: 355 bytes
  ✓ CSV (csv)
    SHA-256: 6e1f53eac20e24d61c5109795a63e63f525ba016a75c425063ff7a59dae1688d
    Tamaño: 278 bytes
  ✓ HTML (html)
    SHA-256: 7afc4a6ba7bcb1d33ba7b082cfd88c60a7c5335ef3939ce599d7c5f8f29e1ced
    Tamaño: 705 bytes


Resumen de Fuentes Procesadas:
Source ID Type  Size (bytes)             SHA-256                  Timestamp
      TXT  txt           355 e04e6b7b2c31aee9... 2026-01-19T00:43:13.562854
      CSV  csv           278 6e1f53eac20e24d6... 2026-01-19T00:43:13.648006
     HTML html           705 7afc4a6ba7bcb1d3... 2026-01-19T00:43:13.718722
